In [1]:
import pandas as pd

Answer to exercises from notebook 2:



`wine.groupby('country')['description'].count().sort_values(ascending=False).plot(kind='bar', figsize=(10,7))
plt.show()`

# Data Cleaning & Manipulation
With Pandas, we can do more than just select data that is already there. 
We can add new columns to our datasets, apply functions, iterate thorugh each row in the dataframe, and more.

This is where we move from "pandas for exploring our data" to "pandas for getting our data ready to feed into models".

In [2]:
wine = pd.read_csv('data/wine_reviews/winemag-data_first150k.csv', index_col=0)
energy = pd.read_csv('data/energy/PJM_Load_hourly.csv', parse_dates=True, index_col=0)

### Dealing with Null Values:

In [4]:
print('Does the wine df conatin nulls?:', wine.isnull().any().any())
print('Does the energy df contain nulls?:', energy.isnull().any().any())

Does the wine df conatin nulls?: True
Does the energy df contain nulls?: False


**Question:** why did we use ".any()" twice when asking this question?

Now, we need to deal with the null values in the wine table.
How we deal with them will depend on the data type of the column in which null values are found, and how we hope to use the data.

If a wine doesn't have a region listed, we can still get a lot of information about it from things like its country. But, if a wine doesn't have a price listed, it's difficult to get much information out of it. 

So, if a wine has a null value for either of its regions, we will set it to be "unknown". But, if it has a null price, we will drop the row.

In [5]:
wine.isnull().any()

country         True
description    False
designation     True
points         False
price           True
province        True
region_1        True
region_2        True
variety        False
winery         False
dtype: bool

In [6]:
print('Shape of wine table before is: ', wine.shape)

null_country = wine[wine['country'].isnull()]
print('Shape of null countries table is:', null_country.shape)

wine = wine[~wine.index.isin(null_country.index)]
print('Shape of wine table after is: ', wine.shape)

Shape of wine table before is:  (150930, 10)
Shape of null countries table is: (5, 10)
Shape of wine table after is:  (150925, 10)


The 5 rows where a NaN value existed for country have been dropped.

Now we'll do the same for rows with a null price.

In [7]:
print('Shape of wine table before is: ', wine.shape)

null_price = wine[wine['price'].isnull()]
print('Shape of null prices table is:', null_price.shape)

wine = wine[~wine.index.isin(null_price.index)]
print('Shape of wine table after is: ', wine.shape)

Shape of wine table before is:  (150925, 10)
Shape of null prices table is: (13695, 10)
Shape of wine table after is:  (137230, 10)


Wow! We lost a lot of rows getting rid of the null price rows.
Can you think of another way we could deal with null prices that doesn't mean losing almost 1/10 of our data? What are the pros and cons to using these other methods?

Now, for the other columns in which there are nulls, we can use `df.fillna()` to populate the null values with a value of our choosing. 

This is where we have to make some decisions. Improperly setting variables can have unintended consequences on any analysis we plan to do later using this data.

In [54]:
wine.isnull().any()

country        False
description    False
designation     True
points         False
price          False
province       False
region_1        True
region_2        True
variety        False
winery         False
dtype: bool

The only columns where we still have nulls are regions and designations.
To avoid having to drop out more rows, we will use "fillna()".

In [55]:
wine.fillna('unknown', inplace=True)

We now have no null values in our dataset!

In [58]:
wine.isnull().any()

country        False
description    False
designation    False
points         False
price          False
province       False
region_1       False
region_2       False
variety        False
winery         False
dtype: bool

In [85]:
wine

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,unknown,Tinta de Toro,Bodega Carmen Rodríguez
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,unknown,Provence red blend,Domaine de la Bégude
5,Spain,"Deep, dense and pure from the opening bell, th...",Numanthia,95,73.0,Northern Spain,Toro,unknown,Tinta de Toro,Numanthia
6,Spain,Slightly gritty black-fruit aromas include a s...,San Román,95,65.0,Northern Spain,Toro,unknown,Tinta de Toro,Maurodos
7,Spain,Lush cedary black-fruit aromas are luxe and of...,Carodorum Único Crianza,95,110.0,Northern Spain,Toro,unknown,Tinta de Toro,Bodega Carmen Rodríguez
8,US,This re-named vineyard was formerly bottled as...,Silice,95,65.0,Oregon,Chehalem Mountains,Willamette Valley,Pinot Noir,Bergström
9,US,The producer sources from two blocks of the vi...,Gap's Crown Vineyard,95,60.0,California,Sonoma Coast,Sonoma,Pinot Noir,Blue Farm


### Creating new columns from existing ones
In pandas, it's easy to make a new column from existing ones. 

In [105]:
wine.loc[:, 'Unique ID'] =  wine.loc[:, 'country'] + "_" + wine.loc[:, 'winery']

#### Using a function + apply to make a new column:
Lets say we want to make a new column for "great, cheap wine".

We will define this as any wine that costs less than $12 and has a rating better than 95.

In [102]:
def helper(row):
    if (row['price'] < 12) & (row['points'] >=95):
        x = 1
    else:
        x = 0
    return x

In [103]:
wine['great and cheap'] = wine.apply(helper, axis=1)

There are NO great and cheap wines?!

In [104]:
wine[wine['great and cheap']==1]

,country,description,designation,points,price,province,region_1,region_2,variety,winery,great and cheap


**Exercise:** Using a function, make a column that indicates if a wine comes from Africa.

**Exercise:** Make a new column in the energy dataset, showing the energy in time T-1 for each time T. Hint: there is a built-in method in Pandas that can help with this, no complicated functions needed :)

**Exercise:** Re-read the CSV for the wine data, and write a function to take care of all the nan values. Bonus points if it includes a test.